Initial Setup

In [3]:
!unzip projects.zip

Archive:  projects.zip
replace projects/simple_calculator/calculator.py? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [10]:
import os
import shutil

def fetch_and_copy_py_files(src_path, dest_path):
    """
    Fetch all Python files from nested folders and copy them to the current directory.

    Parameters:
    src_path (str): The source directory to search for Python files.
    dest_path (str): The destination directory to copy the Python files.
    """
    for root, dirs, files in os.walk(src_path):
        for file in files:
            if file.endswith('.py'):
                full_file_path = os.path.join(root, file)
                shutil.copy(full_file_path, dest_path)
                print(f"Copied: {full_file_path} to {dest_path}")

# Example usage
source_directory = '/content/projects'
destination_directory = '.'  # Current directory
fetch_and_copy_py_files(source_directory, destination_directory)


Copied: /content/projects/file_manager/file_operations.py to .
Copied: /content/projects/data_processor/processor.py to .
Copied: /content/projects/simple_calculator/calculator.py to .


In [8]:
def create_empty_txt_file(file_path):
    with open(file_path, 'w') as file:
        pass  # The pass statement is a no-op; the file is created and left empty.

# Example usage
create_empty_txt_file('analysis_report.txt')

In [11]:
!pip install pynguin
!pip install radon
!pip install pytest
!pip install coverage
!pip install openai==0.28


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.8/307.8 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 30.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.3/276.3 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 339.9/339.9 kB 33.8 MB/s eta 0:00:00
  Created wheel for asciitree: filename=asciitree-0.3.3-py3-none-any.whl size=5034 sha256=b6f130ff1de3abc7db75e1648aba450c7a0b8a07b9fde65005c51403a046693c
  Stored in directory: /root/.cache/pip/wheels/7f/4e/be/1171b40f43b918087657ec57cf3b81fa1a2e027d8755baa184
Successfully built asciitree
  Attempting uninstall: pytest
    Found existing installation: pytest 7.4.4
    Uninstalling pytest-7.4.4:
      Successfully uninstalled pytest-7.4.4
  Attempting uninstall: Pygment

Function to check for Mccabe Complexity of Source Code

In [1]:
import os
from radon.complexity import cc_visit

# Function to get McCabe complexity
def get_mccabe_complexity(file_path):
    with open(file_path, 'r') as file:
        code = file.read()
    complexity_data = cc_visit(code)
    total_complexity = sum(c.complexity for c in complexity_data)
    return total_complexity

# Function to select core modules
def select_core_modules(directory):
    core_modules = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.py'):
                file_path = os.path.join(root, file)
                file_complexity = get_mccabe_complexity(file_path)
                core_modules.append((file_path, file_complexity))

    # Sort by complexity
    core_modules.sort(key=lambda x: -x[1])  # Sort by complexity only
    return core_modules

Normal Generation

In [2]:
import openai
#Use Open AI API KEY
openai.api_key="sk-"
def generate_test_cases(file_path):

    with open(file_path, 'r') as file:
        function_code = file.read()

    prompt = f"""
    The following is a Python program:

    {function_code}

    Write Unit tests using Pytest for given Python code that covers all the edge cases.

    Give Unit tests code only and no additional text such that it can be directly copied for execution without any modification.

    Clearly mention the name of the module to be used take the reference from file path: {file_path}
    ###Do not include this line in generated tests: "```python"
    """

    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7,
        max_tokens=1500,
        top_p=0.9,
    )

    return response.choices[0].message.content.strip()


In [3]:
print(generate_test_cases('file_operations.py'))

import os
import tempfile
from file_operations import FileManager
import pytest

@pytest.fixture
def file_manager():
    base_directory = tempfile.mkdtemp()
    return FileManager(base_directory)

def test_create_file(file_manager):
    file_name = 'test.txt'
    content = 'Hello, World!'
    file_path = file_manager.create_file(file_name, content)
    assert os.path.exists(file_path)
    with open(file_path, 'r') as file:
        assert file.read() == content

def test_read_file(file_manager):
    file_name = 'test.txt'
    content = 'Hello, World!'
    file_manager.create_file(file_name, content)
    assert file_manager.read_file(file_name) == content

def test_delete_file(file_manager):
    file_name = 'test.txt'
    file_manager.create_file(file_name)
    assert file_manager.delete_file(file_name) == True
    assert not os.path.exists(os.path.join(file_manager.base_directory, file_name))
    assert file_manager.delete_file(file_name) == False

def test_list_files(file_manager):
   

# Approach 1 using Few Shot Learning

Create Manual Testing Examples


In [19]:
few_shot_examples = """
import pytest
from calculator import Calculator

def test_add():
    calc = Calculator()
    result = calc.add(5, 7)
    assert result == 12

def test_subtract():
    calc = Calculator()
    result = calc.subtract(10, 5)
    assert result == 5

def test_multiply():
    calc = Calculator()
    result = calc.multiply(3, 7)
    assert result == 21

def test_divide():
    calc = Calculator()
    result = calc.divide(10, 2)
    assert result == 5
"""
def generate_test_cases_fewshot(file_path,few_shot_examples):

    with open(file_path, 'r') as file:
        function_code = file.read()

    prompt = f"""
    The following is a Python program:

    {function_code}

    Here are some examples of unit tests using Pytest:

    {few_shot_examples}

    Write additional unit tests using Pytest for the given Python code that covers all the edge cases.

    Give unit tests code only and no additional text such that it can be directly copied for execution without any modification.

    Clearly mention the name of the module to be used, taking the reference from file path: {file_path}
    ###Do not include this line in generated tests: "```python"
    """

    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7,
        max_tokens=1500,
        top_p=0.9,
    )

    return response.choices[0].message.content.strip()

In [20]:
print(generate_test_cases_fewshot('calculator.py',few_shot_examples))

import pytest
from calculator import Calculator

def test_divide_by_zero():
    calc = Calculator()
    with pytest.raises(ValueError):
        calc.divide(10, 0)

def test_power():
    calc = Calculator()
    result = calc.power(2, 3)
    assert result == 8

def test_power_zero_exponent():
    calc = Calculator()
    result = calc.power(2, 0)
    assert result == 1

def test_power_zero_base():
    calc = Calculator()
    result = calc.power(0, 5)
    assert result == 0

def test_factorial():
    calc = Calculator()
    result = calc.factorial(5)
    assert result == 120

def test_factorial_zero():
    calc = Calculator()
    result = calc.factorial(0)
    assert result == 1

def test_factorial_negative():
    calc = Calculator()
    with pytest.raises(ValueError):
        calc.factorial(-5)


# Approach 2: Use Test Generation from Pynquin and LLM and Combine Results with Iterative Improvement

In [4]:
import subprocess
import os
def generate_test_cases_with_pynguin(module_name):
    """Generate test cases using Pynguin."""
    os.environ['PYNGUIN_DANGER_AWARE'] = 'YES'
    subprocess.run([
        'pynguin',
        '--project-path', './',
        '--output-path', './',
        '--module-name', module_name,
        '-v'
    ])

#generate_test_cases_with_pynguin('calculator')

Check Coverage of Test Cases

In [12]:
# Example usage:
def evaluate_coverage(file_path):

    code_string =   f"""
import os
import coverage
import inspect
import pytest
import json
import subprocess

# Discover and run all test functions in test_calculator module

pytest.main(['-v', '--tb=short', '{file_path}'])
subprocess.run(['coverage', 'run', '-m', 'pytest', '{file_path}'])
subprocess.run(['coverage', 'json', '-o', 'coverage.json'])

# Generate the coverage report
subprocess.run(['coverage', 'report', '-m'])

with open('coverage.json') as f:
        coverage_data = json.load(f)

# Extract the file coverage information
file_coverage = coverage_data['files'].get('{file_path}')
if file_coverage:
    executable = file_coverage['executed_lines']
    executed = file_coverage['executed_lines']
    missing = file_coverage['missing_lines']
    excluded = file_coverage['excluded_lines']
    print(f"File: {file_path}")
    print(f"Executable Lines: {{executable}}")
    print(f"Executed Lines: {{executed}}")
    print(f"Missed Lines: {{missing}}")
    print(f"Excluded Lines: {{excluded}}")
    report_file='analysis_report.txt'
    with open(report_file, 'w') as report:
        report.write(f"Missed lines: {{missing}}")
else:
   print("Failed")
                    """
    filename='run_coverage.py'
    if not filename.endswith('.py'):
            filename += '.py'

    # Write the code string to the file
    with open(filename, 'w') as file:
        file.write(code_string)
    !python run_coverage.py
    #return missed_lines

In [13]:
import re
import os
def get_missed_statements(report_path):
    """
    Extracts missed statements from a coverage report in a text file.

    Parameters:
    report_path (str): The path to the text coverage report file.

    Returns:
    list: A list of missed statements.
    """
    missed_statements = []

    with open(report_path, 'r') as file:
        lines = file.readlines()

    # Regular expression to match the missed lines
    missed_lines_pattern = re.compile(r"Missed lines:\s*\[([0-9,\s]*)\]")

    for line in lines:
        match = missed_lines_pattern.search(line)
        if match:
            missed_lines_str = match.group(1).strip()
            if missed_lines_str:
                missed_lines = [int(x.strip()) for x in missed_lines_str.split(',') if x.strip().isdigit()]
                missed_statements.extend(missed_lines)

    print(missed_statements)
    return missed_statements

def iterative_improvement(file_path, module_name):
    with open(file_path, 'r') as file:
        code = file.read()

    pynguin_test_file = './test_' + module_name.replace('.', '_') + '.py'
    generate_test_cases_with_pynguin(module_name)

    improved_tests = generate_test_cases(file_path)
    mess = "Here are the unit tests using Pytest:"
    if(improved_tests[:len(mess)]==mess):
        improved_tests=improved_tests[len(mess):]

    if(improved_tests[:3]=="```"):
        improved_tests = improved_tests[9:-3]

    openai_test_file = 'test_openai_' + module_name.replace('.', '_') + '.py'

    with open(openai_test_file, 'w') as file:
        file.write(improved_tests)

    combined_test_file = 'combined_tests.py'

    with open(combined_test_file, 'w') as outfile:
        for fname in [pynguin_test_file, openai_test_file]:
            with open(fname) as infile:
                outfile.write(infile.read())
                outfile.write("\n")

    previous_missed_statements = set()
    with open(combined_test_file, 'r') as file:
        testcase_code = file.read()
    improved_tests=testcase_code
    while True:
        evaluate_coverage(combined_test_file)
        new_missed_statements = set(get_missed_statements('analysis_report.txt'))

        if not new_missed_statements or new_missed_statements == previous_missed_statements:
            break

        previous_missed_statements = new_missed_statements

        prompt = f"""
        The following is a Python program:

        {code}

        The current test cases are missing coverage of the above code for the following statements: {missed_statements}.
        Please modify this exsisting testcases that cover these missed statements.
        Exsisting testcases: {testcase_code}

        Clearly mention the name of the module to be used take the reference from file path: {file_path}

        Do not give text apart from code. Note it is very important that you give code only do not generate anything else apart from code.
        """
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7,
            max_tokens=1500,
            top_p=0.9,
        )


        improved_tests = response.choices[0].message.content.strip()
        if(improved_tests[:65]=="Here is the unit test code that covers all the missed statements:"):
            improved_tests=improved_tests[65:]
        if(improved_tests[:3]=="```"):
            improved_tests=improved_tests[9:-3]
        filename=combined_test_file

        with open(filename, 'w') as file:
            file.write(improved_tests)


    return improved_tests


In [14]:
improved_tests = iterative_improvement('processor.py','processor')
print(improved_tests)

======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-8.2.2, pluggy-1.5.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content
plugins: anyio-3.7.1
collected 13 items                                                                                 

combined_tests.py::test_case_0 XFAIL                                                         [  7%]
combined_tests.py::test_case_1 PASSED                                                        [ 15%]
combined_tests.py::test_case_2 XFAIL                                                         [ 23%]
combined_tests.py::test_case_3 XFAIL                                                         [ 30%]
combined_tests.py::test_case_4 XFAIL                                                         [ 38%]
combined_tests.py::test_case_5 XFAIL                                                         [ 46%]
combined_tests.py::test_case_6 XFAIL                  

# Approach 3: Reasoning Generation and Review (Used to Identify the logic behind test cases and Help in improving them)

In [15]:
def generate_tests_with_reasoning(file_path):

    with open(file_path, 'r') as file:
        function_code = file.read()

    prompt = f"""
    The following is a Python program:

    {function_code}

    Write Unit tests using Pytest with Reasoning for given Python code that covers all the edge cases.
    Give Unit tests only and Reasoning and no additional text.

    Clearly mention the name of the module to be used take the reference from file path: {file_path}
    ###Do not include this line in generated tests: "```python
    """
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7,
        max_tokens=800,
        top_p=0.9,
    )

    return response.choices[0].message.content.strip()


In [16]:
print(generate_tests_with_reasoning('file_operations.py'))

```python
import os
import pytest
from file_operations import FileManager


def test_create_file(tmpdir):
    base_dir = tmpdir.mkdir("sub")
    file_manager = FileManager(base_dir)
    file_name = "test.txt"
    content = "This is a test file."

    # Test that the file is created in the correct location with the correct content
    assert file_manager.create_file(file_name, content) == os.path.join(base_dir, file_name)
    with open(os.path.join(base_dir, file_name), 'r') as file:
        assert file.read() == content

def test_read_file(tmpdir):
    base_dir = tmpdir.mkdir("sub")
    file_manager = FileManager(base_dir)
    file_name = "test.txt"
    content = "This is a test file."

    # Create a file to read
    with open(os.path.join(base_dir, file_name), 'w') as file:
        file.write(content)

    # Test that the content of the file is read correctly
    assert file_manager.read_file(file_name) == content

def test_delete_file(tmpdir):
    base_dir = tmpdir.mkdir("sub")
    

# Approach 4: Iterative Improvement till Complete Coverage of Source Code by Unit Tests

In [17]:
import re
import os
def get_missed_statements(report_path):
    """
    Extracts missed statements from a coverage report in a text file.

    Parameters:
    report_path (str): The path to the text coverage report file.

    Returns:
    list: A list of missed statements.
    """
    missed_statements = []

    with open(report_path, 'r') as file:
        lines = file.readlines()

    # Regular expression to match the missed lines
    missed_lines_pattern = re.compile(r"Missed lines:\s*\[([0-9,\s]*)\]")

    for line in lines:
        match = missed_lines_pattern.search(line)
        if match:
            missed_lines_str = match.group(1).strip()
            if missed_lines_str:
                missed_lines = [int(x.strip()) for x in missed_lines_str.split(',') if x.strip().isdigit()]
                missed_statements.extend(missed_lines)

    print(missed_statements)
    return missed_statements


def iterative_improvement(file_path):
    with open(file_path, 'r') as file:
        code = file.read()

    missed_statements = []
    improved_tests = generate_test_cases(file_path)
    if(improved_tests[:65]=="Here is the unit test code that covers all the missed statements:"):
        improved_tests=improved_tests[65:]
    if(improved_tests[:3]=="```"):
        improved_tests=improved_tests[9:-3]
    filename=f'test_{os.path.basename(file_path)}'
    if not filename.endswith('.py'):
            filename += '.py'

    # Write the code string to the file
    print(filename)
    with open(filename, 'w') as file:
        file.write(improved_tests)



    while True:
        evaluate_coverage(filename)
        new_missed_statements = get_missed_statements('analysis_report.txt')  # Implement this function based on your coverage tool

        if not new_missed_statements or new_missed_statements == missed_statements:
            break

        missed_statements = new_missed_statements
        prompt = f"""
        The following is a Python program:

        {code}

        The current test cases are missing coverage for the following statements: {missed_statements}.
        Please generate new unit tests using Pytest that cover these missed statements.

        Clearly mention the name of the module to be used take the reference from file path: {file_path}

        Do not give text apart from code. Note it is very important that you give code only do not generate anything else apart from code.
        """
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7,
            max_tokens=1500,
            top_p=0.9,
        )


        improved_tests=response.choices[0].message.content.strip()
        if(improved_tests[:65]=="Here is the unit test code that covers all the missed statements:"):
            improved_tests=improved_tests[65:]
        if(improved_tests[:3]=="```"):
            improved_tests=improved_tests[9:-3]

        with open(filename, 'w') as file:
            file.write(improved_tests)

    return improved_tests




In [18]:
# Example usage
improved_tests = iterative_improvement('file_operations.py')
print(improved_tests)

test_file_operations.py
======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-8.2.2, pluggy-1.5.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content
plugins: anyio-3.7.1
collected 5 items                                                                                  

test_file_operations.py::test_create_file PASSED                                             [ 20%]
test_file_operations.py::test_read_file PASSED                                               [ 40%]
test_file_operations.py::test_delete_file PASSED                                             [ 60%]
test_file_operations.py::test_delete_non_existent_file PASSED                                [ 80%]
test_file_operations.py::test_list_files PASSED                                              [100%]

======================================== 5 passed in 0.03s =========================================
============================